# Settings

In [1]:
%env TF_KERAS = 1
import os
sep_local = os.path.sep

import sys
sys.path.append('..'+sep_local+'..')
print(sep_local)

env: TF_KERAS=1
\


In [2]:
import tensorflow as tf
print(tf.__version__)

2.1.0


# Dataset loading

In [3]:
dataset_name='pokemon'

In [4]:
IMG_DIR = 'C:\\projects\\pokemon\DS06\\'
VAL_PCT = 10
VAL_FORMAT = 'png'

In [5]:
from training.generators.file_image_generator import create_image_lists, get_generators

Using TensorFlow backend.


In [6]:
imgs_list = create_image_lists(
    image_dir=IMG_DIR, 
    validation_pct=VAL_PCT, 
    valid_imgae_formats=VAL_FORMAT
)

  DEBUG    | Looking for images in 'all'


  INFO     | 809 file found


In [7]:
inputs_shape= IMG_SIZE=(80, 80, 3)
BATCH_SIZE = 32
latent_dim = 100
intermediate_dim = 100
TRAIN_BUF = 600
TEST_BUF = 100

In [8]:
train_gen, test_gen = get_generators(
    images_list=imgs_list, 
    image_dir=IMG_DIR, 
    image_size=IMG_SIZE, 
    batch_size=BATCH_SIZE, 
    class_mode=None
)

  INFO     | Found 738 training files


  INFO     | Found 71 validation files


In [9]:
import tensorflow as tf

In [10]:
train_ds = tf.data.Dataset.from_generator(
    lambda: train_gen, 
    output_types=tf.float32 ,
    output_shapes=tf.TensorShape((BATCH_SIZE, ) + IMG_SIZE)
)


In [11]:
_instance_scale=1.0
for data in train_ds:
    _instance_scale = float(data[0].numpy().max())
    break

In [12]:
_instance_scale

1.0

In [13]:
import numpy as np
from collections.abc import Iterable

In [14]:
if isinstance(inputs_shape, Iterable):
    _outputs_shape = np.prod(inputs_shape)

In [15]:
_outputs_shape

19200

# Model's Layers definition

In [16]:
enc_lays = [tf.keras.layers.Dense(units=intermediate_dim, activation='relu'),
            tf.keras.layers.Dense(units=intermediate_dim, activation='relu'),
            tf.keras.layers.Flatten(),
            tf.keras.layers.Dense(units=latent_dim)]

dec_lays = [tf.keras.layers.Dense(units=latent_dim, activation='relu'),
            tf.keras.layers.Dense(units=intermediate_dim, activation='relu'),
            tf.keras.layers.Dense(units=_outputs_shape),
            tf.keras.layers.Reshape(inputs_shape)]

# Model definition

In [17]:
model_name = dataset_name+'AEpsnr'
recoding_dir='..'+sep_local+'..'+sep_local+'recoding'+sep_local+model_name

In [18]:
from training.traditional.autoencoders.autoencoder import autoencoder as AE

In [19]:
inputs_shape=IMG_SIZE

In [20]:
variables_params = \
[
    {
        'name': 'inference', 
        'inputs_shape':inputs_shape,
        'outputs_shape':latent_dim,
        'layers': enc_lays#enc_lays2#
    }

    ,
    
        {
        'name': 'generative', 
        'inputs_shape':latent_dim,
        'outputs_shape':inputs_shape,
        'layers':dec_lays#dec_lays2#
    }
]

In [21]:
from utils.data_and_files.file_utils import create_if_not_exist

In [22]:
_restore = os.path.join(recoding_dir, 'var_save_dir')

In [23]:
create_if_not_exist(_restore)
_restore

'..\\..\\recoding\\pokemonAEpsnr\\var_save_dir'

In [24]:
#to restore trained model, set filepath=_restore

In [25]:
ae = AE( 
    name=model_name,
    inputs_shape=inputs_shape,
    outputs_shape=inputs_shape,
    latent_dim=latent_dim,
    batch_size=BATCH_SIZE,
    variables_params=variables_params, 
    filepath=None
    )

Model: "inference"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
inference_inputs (InputLayer [(None, 80, 80, 3)]       0         
_________________________________________________________________
dense (Dense)                (None, 80, 80, 100)       400       
_________________________________________________________________
dense_1 (Dense)              (None, 80, 80, 100)       10100     
_________________________________________________________________
flatten (Flatten)            (None, 640000)            0         
_________________________________________________________________
dense_2 (Dense)              (None, 100)               64000100  
_________________________________________________________________
batch_normalization (BatchNo (None, 100)               400       
_________________________________________________________________
dropout (Dropout)            (None, 100)               0 

  WARNING  | None


Model: "generative"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
generative_inputs (InputLaye [(None, 100)]             0         
_________________________________________________________________
dense_3 (Dense)              (None, 100)               10100     
_________________________________________________________________
dense_4 (Dense)              (None, 100)               10100     
_________________________________________________________________
dense_5 (Dense)              (None, 19200)             1939200   
_________________________________________________________________
reshape (Reshape)            (None, 80, 80, 3)         0         
_________________________________________________________________
batch_normalization_1 (Batch (None, 80, 80, 3)         12        
_________________________________________________________________
dropout_1 (Dropout)          (None, 80, 80, 3)         0

  WARNING  | None


In [26]:
from evaluation.quantitive_metrics.psnr import psnr
#from evaluation.quantitive_metrics.ssmi import ssmi
#from evaluation.quantitive_metrics.sharp_diff import sharp_diff


In [27]:
ae.compile(loss={'x_logits': psnr})

Model: "pokemonAEpsnr"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
inference_inputs (InputLayer [(None, 80, 80, 3)]       0         
_________________________________________________________________
inference (Model)            (None, 100)               64011000  
_________________________________________________________________
generative (Model)           (None, 80, 80, 3)         1959412   
_________________________________________________________________
tf_op_layer_x_logits (Tensor [(None, 80, 80, 3)]       0         
Total params: 65,970,412
Trainable params: 65,970,206
Non-trainable params: 206
_________________________________________________________________
None


# Callbacks

In [28]:
from training.callbacks.progress_bar import NotebookPrograssBar
from training.callbacks.sample_generation import SampleGeneration
from training.callbacks.save_model import ModelSaver

In [29]:
progbar = NotebookPrograssBar(leave_outer=False)

In [30]:
es = tf.keras.callbacks.EarlyStopping(
    monitor='loss', 
    min_delta=1e-12, 
    patience=5, 
    verbose=1, 
    restore_best_weights=True
)

In [31]:
ms = ModelSaver(filepath=_restore,save_freq=5)

In [32]:
csv_dir = os.path.join(recoding_dir, 'csv_dir')
create_if_not_exist(csv_dir)
csv_dir = os.path.join(csv_dir, ae.name+'.csv')
csv_log = tf.keras.callbacks.CSVLogger(csv_dir, append=True)
csv_dir

'..\\..\\recoding\\pokemonAEpsnr\\csv_dir\\pokemonAEpsnr.csv'

In [33]:
image_gen_dir = os.path.join(recoding_dir, 'image_gen_dir')
create_if_not_exist(image_gen_dir)

In [34]:
sg = SampleGeneration(latent_shape=latent_dim, filepath=image_gen_dir, gen_freq=5, save_img=True, gray_plot=False)

In [35]:
import numpy as np

# Model Training

In [36]:
ae.fit(
    x=train_ds,
    input_kw=None,
    steps_per_epoch=50,
    epochs=100, 
    verbose=0,
    callbacks=[progbar, es, ms, csv_log, sg],
    workers=-1,
    use_multiprocessing=True
)